In [ ]:
DATASET_ZIP_PATH = 'gs://my-gan-dataset-bucket/gan-getting-started.zip'  # Replace with your actual bucket name

!gsutil cp $DATASET_ZIP_PATH ./input/gan-getting-started.zip

Copying gs://my-gan-dataset-bucket/gan-getting-started.zip...
\ [1 files][367.0 MiB/367.0 MiB]                                                
Operation completed over 1 objects/367.0 MiB.                                    


In [ ]:
import zipfile
import os

# Ensure the input directory exists
INPUT_DIR = './input'
os.makedirs(INPUT_DIR, exist_ok=True)

# Unzip the dataset
with zipfile.ZipFile(os.path.join(INPUT_DIR, 'gan-getting-started.zip'), 'r') as zip_ref:
    zip_ref.extractall(INPUT_DIR)

print("Dataset downloaded and extracted.")


Dataset downloaded and extracted.


In [ ]:
import os

INPUT_DIR = './input/gan-getting-started'
if os.path.exists(INPUT_DIR):
    for root, dirs, files in os.walk(INPUT_DIR):
        print(f"Directory: {root}")
        for name in files:
            print(f"File: {name}")
else:
    print("The input directory does not exist.")


Directory: ./input/gan-getting-started
Directory: ./input/gan-getting-started/photo_jpg
File: cf32b9f4e9.jpg
File: 5f477a5c46.jpg
File: 5619b15af5.jpg
File: ff46474c6d.jpg
File: c0876f3b2b.jpg
File: 03fd7067c1.jpg
File: bb64125bc2.jpg
File: e53b41415d.jpg
File: 4d1755ab04.jpg
File: 9f9c46fa6b.jpg
File: 45db826465.jpg
File: f088b513bf.jpg
File: ec74224ef5.jpg
File: c0511e5414.jpg
File: cd06b978fe.jpg
File: bf8f3d28ae.jpg
File: bd76d694d4.jpg
File: a6e2dd35f8.jpg
File: 01ae8be57e.jpg
File: 153bb660da.jpg
File: 71cde4e6b9.jpg
File: 3db8bf2299.jpg
File: 32f3055dff.jpg
File: 101a1f34db.jpg
File: 0a1d1b9f8e.jpg
File: 7b58d64ffe.jpg
File: 2688ed2270.jpg
File: 94cf38462b.jpg
File: ec6c041436.jpg
File: 6e656030a0.jpg
File: 6157aa63b5.jpg
File: d9213433cf.jpg
File: ffc5b52a77.jpg
File: f6aa1d5154.jpg
File: fe00c3d087.jpg
File: e86d465a2d.jpg
File: a5bc67adcd.jpg
File: 05f0db4d14.jpg
File: ef674070d3.jpg
File: 2f8ca8f7dd.jpg
File: 67b303bc1f.jpg
File: 5c399c47f2.jpg
File: 748d5a0ab8.jpg
File: ed9

In [ ]:
import zipfile
import os

# Path to downloaded zip file
DATASET_ZIP_PATH = './input/gan-getting-started.zip'  # Adjust path if needed
EXTRACT_DIR = './input/gan-getting-started'

# Ensure the extraction directory exists
os.makedirs(EXTRACT_DIR, exist_ok=True)

# Unzip the dataset
with zipfile.ZipFile(DATASET_ZIP_PATH, 'r') as zip_ref:
    zip_ref.extractall(EXTRACT_DIR)

print("Dataset extracted successfully.")


Dataset extracted successfully.


In [ ]:
import shutil

TEMP_DIR = './working/temp_images'
OUTPUT_DIR = './working/generated_images'

# Ensure output directories exist
os.makedirs(TEMP_DIR, exist_ok=True)
os.makedirs(OUTPUT_DIR, exist_ok=True)

temp_monet_dir = os.path.join(TEMP_DIR, 'monet')
temp_photo_dir = os.path.join(TEMP_DIR, 'photo')

# Create subdirectories for monet and photo
os.makedirs(temp_monet_dir, exist_ok=True)
os.makedirs(temp_photo_dir, exist_ok=True)

# Copy images to a writable directory
for img in os.listdir(monet_images_dir):
    if img.endswith(".jpg"):
        shutil.copy(os.path.join(monet_images_dir, img), os.path.join(temp_monet_dir, img))

for img in os.listdir(photo_images_dir):
    if img.endswith(".jpg"):
        shutil.copy(os.path.join(photo_images_dir, img), os.path.join(temp_photo_dir, img))

print("Images copied to writable directory.")



Images copied to writable directory.


In [ ]:
import os
import zipfile
import shutil
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, LeakyReLU, BatchNormalization
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import zipfile

# Load and preprocess the dataset
def load_images_from_directory(image_dir, img_size=(256, 256), batch_size=32):
    datagen = ImageDataGenerator(rescale=1./127.5)  # Scale images to [-1, 1]
    generator = datagen.flow_from_directory(
        image_dir,
        target_size=img_size,
        batch_size=batch_size,
        class_mode=None,
        shuffle=True
    )
    return generator

train_monet_generator = load_images_from_directory(TEMP_DIR, img_size=(256, 256), batch_size=32)
train_photo_generator = load_images_from_directory(TEMP_DIR, img_size=(256, 256), batch_size=32)

# Set image dimensions
IMG_SIZE = (256, 256)
batch_size = 32
H, W, C = IMG_SIZE[0], IMG_SIZE[1], 3  # Height, width, and channels (RGB)
D = H * W * C

# Function to flatten images
def flatten_images(img_batch):
    if img_batch.shape[0] > 0:
        return img_batch.reshape(img_batch.shape[0], D)
    else:
        raise ValueError("No images found in batch.")

# GAN Architecture
latent_dim = 100

# Generator Model
def build_generator(latent_dim):
    i = Input(shape=(latent_dim,))
    x = Dense(256, activation=LeakyReLU(alpha=0.2))(i)
    x = BatchNormalization(momentum=0.7)(x)
    x = Dense(512, activation=LeakyReLU(alpha=0.2))(x)
    x = BatchNormalization(momentum=0.7)(x)
    x = Dense(1024, activation=LeakyReLU(alpha=0.2))(x)
    x = BatchNormalization(momentum=0.7)(x)
    x = Dense(D, activation='tanh')(x)
    model = Model(i, x)
    return model

# Discriminator Model
def build_discriminator(img_size):
    i = Input(shape=(img_size,))
    x = Dense(512, activation=LeakyReLU(alpha=0.2))(i)
    x = Dense(256, activation=LeakyReLU(alpha=0.2))(x)
    x = Dense(1, activation='sigmoid')(x)
    model = Model(i, x)
    return model

# Build and compile the discriminator
discriminator = build_discriminator(D)
discriminator.compile(
    loss='binary_crossentropy',
    optimizer=Adam(0.0002, 0.5),
    metrics=['accuracy']
)

# Build the generator
generator = build_generator(latent_dim)

# Freeze discriminator's weights when training the combined model
discriminator.trainable = False

# Combined model (generator + discriminator)
noise_input = Input(shape=(latent_dim,))
fake_img = generator(noise_input)
fake_pred = discriminator(fake_img)
combined_model = Model(noise_input, fake_pred)
combined_model.compile(loss='binary_crossentropy', optimizer=Adam(0.0002, 0.5))

# Training Parameters
epochs = 30000
sample_period = 200
ones = np.ones(batch_size)
zeros = np.zeros(batch_size)

# Track losses
d_losses = []
g_losses = []

# Training loop
for epoch in range(epochs):
    ###########################
    ### Train discriminator ###
    discriminator.trainable = True  # Unfreeze the discriminator's weights

    # Get real images and adjust the label array dynamically
    real_imgs = next(train_monet_generator)
    real_imgs = flatten_images(real_imgs)
    batch_size_actual = real_imgs.shape[0]  # Actual batch size from generator

    # Adjust `ones` and `zeros` to match the actual batch size
    ones = np.ones(batch_size_actual)
    zeros = np.zeros(batch_size_actual)

    # Generate fake images
    noise = np.random.randn(batch_size_actual, latent_dim)
    fake_imgs = generator.predict(noise)

    # Train on real and fake images
    d_loss_real, d_acc_real = discriminator.train_on_batch(real_imgs, ones)
    d_loss_fake, d_acc_fake = discriminator.train_on_batch(fake_imgs, zeros)
    d_loss = 0.5 * (d_loss_real + d_loss_fake)
    d_acc = 0.5 * (d_acc_real + d_acc_fake)

    #######################
    ### Train generator ###
    discriminator.trainable = False  # Freeze discriminator for generator training

    noise = np.random.randn(batch_size, latent_dim)  # Use original batch size for generator
    g_loss = combined_model.train_on_batch(noise, np.ones(batch_size))

    # Track generator loss
    g_loss_value = g_loss if not isinstance(g_loss, list) else g_loss[0]

    # Print progress every few epochs
    if epoch % sample_period == 0:
        print(f"epoch: {epoch+1}/{epochs}, d_loss: {d_loss:.2f}, d_acc: {d_acc:.2f}, g_loss: {g_loss_value:.2f}")

# Generate and save images
def generate_monet_style_images(generator, output_dir=OUTPUT_DIR, zip_name='images.zip', max_images=10000):
    image_count = 0
    while image_count < max_images:
        noise = np.random.randn(batch_size, latent_dim)
        generated_imgs = generator.predict(noise)
        generated_imgs = 0.5 * generated_imgs + 0.5  # Rescale to [0, 1]
        generated_imgs = (generated_imgs * 255).astype(np.uint8)
        for j in range(generated_imgs.shape[0]):
            img = generated_imgs[j].reshape(H, W, C)
            img_path = os.path.join(output_dir, f'generated_image_{image_count}.jpg')
            plt.imsave(img_path, img)
            image_count += 1
            if image_count >= max_images:
                break

    # Zip images
    zip_path = os.path.join(WORKING_DIR, zip_name)
    with zipfile.ZipFile(zip_path, 'w') as z:
        for foldername, _, filenames in os.walk(output_dir):
            for filename in filenames:
                z.write(os.path.join(foldername, filename), arcname=filename)
    print(f'Generated images saved in {zip_name}')

# Generate images after training
generate_monet_style_images(generator, output_dir=OUTPUT_DIR, zip_name='images.zip', max_images=10000)

# Plot the losses
plt.plot(g_losses, label='g_losses')
plt.plot(d_losses, label='d_losses')
plt.legend()
plt.show()


Found 7338 images belonging to 2 classes.
Found 7338 images belonging to 2 classes.
1/1 [==============================] - 0s 143ms/step
epoch: 1/30000, d_loss: 0.48, d_acc: 0.56, g_loss: 0.69
1/1 [==============================] - 0s 62ms/step
epoch: 201/30000, d_loss: 26.56, d_acc: 0.59, g_loss: 19.95
1/1 [==============================] - 0s 63ms/step
epoch: 401/30000, d_loss: 9.24, d_acc: 0.50, g_loss: 0.16
1/1 [==============================] - 0s 61ms/step
epoch: 601/30000, d_loss: 1.82, d_acc: 0.31, g_loss: 0.04
1/1 [==============================] - 0s 62ms/step
epoch: 801/30000, d_loss: 0.37, d_acc: 0.70, g_loss: 0.94
1/1 [==============================] - 0s 62ms/step
epoch: 1001/30000, d_loss: 2.53, d_acc: 0.50, g_loss: 0.09
1/1 [==============================] - 0s 64ms/step
epoch: 1201/30000, d_loss: 0.37, d_acc: 0.72, g_loss: 1.27
1/1 [==============================] - 0s 63ms/step
epoch: 1401/30000, d_loss: 3.97, d_acc: 0.52, g_loss: 0.04
1/1 [===========================